In [1]:
import numpy as np, pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import minimize
import scipy.stats as stats
import sympy as sp
import itertools
from itertools import permutations 
from itertools import chain
from itertools import product
import math as m
from joblib import Parallel, delayed
import multiprocessing

In [2]:
p = sp.symbols('p')

trans_matrix = []
with open('trans_matrix_pref_distal.txt','r') as data_file: 
    for i in data_file: 
        trans_matrix.append(eval(i))
data_file.close()

In [3]:
A1,A2,B1,B2= sp.symbols('A1,A2,B1,B2') 
G,g = sp.symbols('G,g')

In [4]:
geno = [[[i11,i12,i13],[i21,i22,i23],[i31,i32,i33],[i41,i42,i43]] \
                                                         for i11 in [A1,A2] for i12 in [B1,B2] for i13 in [G,g] \
                                                        for i21 in [A1,A2] for i22 in [B1,B2] for i23 in [G,g] \
                                                        for i31 in [A1,A2] for i32 in [B1,B2] for i33 in [G,g] \
                                                        for i41 in [A1,A2] for i42 in [B1,B2] for i43 in [G,g]]

In [5]:
u_geno = [geno[0]]
for item in geno:
    flag = 0
    for perm in itertools.permutations(item):
        if list(perm) in u_geno: 
            flag = 1
    if flag == 0:
        u_geno.append(item)

In [6]:
gam = [[[i11,i12,i13],[i21,i22,i23]] \
                                                         for i11 in [A1,A2] for i12 in [B1,B2] for i13 in [G,g] \
                                                        for i21 in [A1,A2] for i22 in [B1,B2] for i23 in [G,g]]

In [7]:
u_gam = [gam[0]]
for item in gam:
    flag = 0
    for perm in itertools.permutations(item):
        if list(perm) in u_gam: 
            flag = 1
    if flag == 0:
        u_gam.append(item)

In [8]:
import sympy as sp 
   
r= sp.symbols('r')
q=sp.symbols('q')
v=sp.symbols('v')
vp=sp.symbols('vp')
rp=sp.symbols('rp')
qp=sp.symbols('qp')
t=sp.symbols('t')
(ppc)=sp.symbols('(ppc)')
(pdm)=sp.symbols('(pdm)')
(pmp)=sp.symbols('(pmp)')

p_pair = sp.symbols('p_pair')

In [9]:
modes = [[['i_m', 'i_m', G], ['i_m', 'i_m', G]],
 [['i_m', 'j_m', G], ['i_m', 'j_m', G]],
 [['i_m', 'i_m', G], ['i_m', 'j_m', G]],
 [['i_m', 'j_m', G], ['i_m', 'k_m', G]],
 [['i_m', 'i_m', G], ['j_m', 'i_m', G]],
 [['i_m', 'j_m', G], ['k_m', 'j_m', G]],
 [['i_m', 'i_m', G], ['j_m', 'j_m', G]],
 [['i_m', 'i_m', G], ['j_m', 'k_m', G]],
 [['i_m', 'j_m', G], ['j_m', 'i_m', G]],
 [['i_m', 'j_m', G], ['j_m', 'k_m', G]],
 [['i_m', 'j_m', G], ['k_m', 'l_m', G]],
 [['i_m', 'i_m', G], ['i_m', 'j_m', g]],
 [['i_m', 'i_m', G], ['j_m', 'i_m', g]],
 [['i_m', 'i_m', G], ['j_m', 'k_m', g]],
 [['i_m', 'j_m', G], ['j_m', 'k_m', g]],
 [['i_m', 'i_m', g], ['i_m', 'i_m', G]],
 [['i_m', 'j_m', g], ['i_m', 'j_m', G]],
 [['i_m', 'i_m', g], ['i_m', 'j_m', G]],
 [['i_m', 'j_m', g], ['i_m', 'k_m', G]],
 [['i_m', 'i_m', g], ['j_m', 'i_m', G]],
 [['i_m', 'j_m', g], ['k_m', 'j_m', G]],
 [['i_m', 'i_m', g], ['j_m', 'j_m', G]],
 [['i_m', 'i_m', g], ['j_m', 'k_m', G]],
 [['i_m', 'j_m', g], ['j_m', 'i_m', G]],
 [['i_m', 'j_m', g], ['j_m', 'k_m', G]],
 [['i_m', 'j_m', g], ['k_m', 'l_m', G]],
 [['i_m', 'i_m', g], ['i_m', 'i_m', g]],
 [['i_m', 'j_m', g], ['i_m', 'j_m', g]],
 [['i_m', 'i_m', g], ['i_m', 'j_m', g]],
 [['i_m', 'j_m', g], ['i_m', 'k_m', g]],
 [['i_m', 'i_m', g], ['j_m', 'i_m', g]],
 [['i_m', 'j_m', g], ['k_m', 'j_m', g]],
 [['i_m', 'i_m', g], ['j_m', 'j_m', g]],
 [['i_m', 'i_m', g], ['j_m', 'k_m', g]],
 [['i_m', 'j_m', g], ['j_m', 'i_m', g]],
 [['i_m', 'j_m', g], ['j_m', 'k_m', g]],
 [['i_m', 'j_m', g], ['k_m', 'l_m', g]]]


In [10]:
mode_prob = [[0,
  [-(ppc)*t*vp*(q - 1)**2*(vp - 1)/2,
   (ppc)*t*vp*(-2*p_pair**6*q**2*r**2*vp + 2*p_pair**5*q**2*r**2 + 2*p_pair**5*q**2*r*vp + 4*p_pair**5*q*r**2*vp - 2*p_pair**4*q**2*r - p_pair**4*q**2*vp - 4*p_pair**4*q*r**2 - 4*p_pair**4*q*r*vp - 2*p_pair**4*r**2*vp + p_pair**3*q**2 + 4*p_pair**3*q*r + 2*p_pair**3*q*vp + 2*p_pair**3*r**2 + 2*p_pair**3*r*vp - 2*p_pair**2*q - 2*p_pair**2*r - p_pair**2*vp + p_pair - 2*q**2*vp + 2*q**2 + 4*q*vp - 4*q - 2*vp + 2)/8,
   (ppc)*t*vp*(-4*p_pair**6*q**2*r**2*vp + 4*p_pair**5*q**2*r**2 + 4*p_pair**5*q**2*r*vp + 8*p_pair**5*q*r**2*vp - 4*p_pair**4*q**2*r - 2*p_pair**4*q**2*vp - 8*p_pair**4*q*r**2 - 8*p_pair**4*q*r*vp - 4*p_pair**4*r**2*vp + 2*p_pair**3*q**2 + 8*p_pair**3*q*r + 4*p_pair**3*q*vp + 4*p_pair**3*r**2 + 4*p_pair**3*r*vp - 4*p_pair**2*q - 4*p_pair**2*r - 2*p_pair**2*vp + 2*p_pair - q**2*vp + q**2 + 2*q*vp - 2*q - vp + 1)/12,
   (ppc)*t*vp*(-p_pair**4*q**2*vp + p_pair**3*q**2 + 2*p_pair**3*q*vp - 2*p_pair**2*q - p_pair**2*vp + p_pair - 2*q**2*vp + 2*q**2 + 4*q*vp - 4*q - 2*vp + 2)/8]],
 [1,
  [(ppc)*q**2*t*vp*(-vp + 1)/2,
   (ppc)*q**2*t*vp*(-2*p_pair**6*r**2*vp + 2*p_pair**5*r**2 + 2*p_pair**5*r*vp - 2*p_pair**4*r - p_pair**4*vp + p_pair**3 - 2*vp + 2)/8,
   (ppc)*q**2*t*vp*(-4*p_pair**6*r**2*vp + 4*p_pair**5*r**2 + 4*p_pair**5*r*vp - 4*p_pair**4*r - 2*p_pair**4*vp + 2*p_pair**3 - vp + 1)/12,
   (ppc)*q**2*t*vp*(-p_pair**4*vp + p_pair**3 - 2*vp + 2)/8]],
 [2,
  [(ppc)*q*t*vp*(q - 1)*(vp - 1),
   (ppc)*q*t*vp*(-p_pair**6*q*r**2*vp + p_pair**5*q*r**2 + p_pair**5*q*r*vp + p_pair**5*r**2*vp - p_pair**4*q*r - p_pair**4*r**2 - p_pair**4*r*vp + p_pair**3*r + q*vp - q - vp + 1)/2,
   (ppc)*q*t*vp*(-4*p_pair**6*q*r**2*vp + 4*p_pair**5*q*r**2 + 4*p_pair**5*q*r*vp + 4*p_pair**5*r**2*vp - 4*p_pair**4*q*r - 4*p_pair**4*r**2 - 4*p_pair**4*r*vp + 4*p_pair**3*r + q*vp - q - vp + 1)/6,
   (ppc)*q*t*vp*(p_pair**4*q*vp - p_pair**3*q - p_pair**3*vp + p_pair**2 + 2*q*vp - 2*q - 2*vp + 2)/4]],
 [3, [0, 0, 0, 0]],
 [4,
  [t*((pmp)*qp*(qp - 1)**2 - (ppc)*q*(q - 1)*(2*vp**2 - 2*vp + 1))/2,
   t*((pmp)*qp*(qp - 1)*(2*p_pair**4*qp*r**2 - 2*p_pair**3*qp*r - 2*p_pair**3*r**2 + p_pair**2*qp + 2*p_pair**2*r - p_pair + 2*qp - 2) - (ppc)*q*(4*p_pair**6*q*r**2*vp**2 - 4*p_pair**5*q*r**2*vp - 4*p_pair**5*q*r*vp**2 - 4*p_pair**5*r**2*vp**2 + 2*p_pair**4*q*r**2 + 4*p_pair**4*q*r*vp + 2*p_pair**4*q*vp**2 + 4*p_pair**4*r**2*vp + 4*p_pair**4*r*vp**2 - 2*p_pair**3*q*r - 2*p_pair**3*q*vp - 2*p_pair**3*r**2 - 4*p_pair**3*r*vp - 2*p_pair**3*vp**2 + p_pair**2*q + 2*p_pair**2*r + 2*p_pair**2*vp - p_pair + 4*q*vp**2 - 4*q*vp + 2*q - 4*vp**2 + 4*vp - 2))/8,
   t*((pmp)*qp*(qp - 1)*(4*p_pair**4*qp*r**2 - 4*p_pair**3*qp*r - 4*p_pair**3*r**2 + 2*p_pair**2*qp + 4*p_pair**2*r - 2*p_pair + qp - 1) - (ppc)*q*(8*p_pair**6*q*r**2*vp**2 - 8*p_pair**5*q*r**2*vp - 8*p_pair**5*q*r*vp**2 - 8*p_pair**5*r**2*vp**2 + 4*p_pair**4*q*r**2 + 8*p_pair**4*q*r*vp + 4*p_pair**4*q*vp**2 + 8*p_pair**4*r**2*vp + 8*p_pair**4*r*vp**2 - 4*p_pair**3*q*r - 4*p_pair**3*q*vp - 4*p_pair**3*r**2 - 8*p_pair**3*r*vp - 4*p_pair**3*vp**2 + 2*p_pair**2*q + 4*p_pair**2*r + 4*p_pair**2*vp - 2*p_pair + 2*q*vp**2 - 2*q*vp + q - 2*vp**2 + 2*vp - 1))/12,
   t*((pmp)*qp*(qp - 1)*(p_pair**2*qp - p_pair + 2*qp - 2) - (ppc)*q*(2*p_pair**4*q*vp**2 - 2*p_pair**3*q*vp - 2*p_pair**3*vp**2 + p_pair**2*q + 2*p_pair**2*vp - p_pair + 4*q*vp**2 - 4*q*vp + 2*q - 4*vp**2 + 4*vp - 2))/8]],
 [5,
  [(pmp)*qp**2*t*(-qp + 1)/2,
   (pmp)*qp**2*t*(-2*p_pair**4*qp*r**2 + 2*p_pair**3*qp*r + 2*p_pair**3*r**2 - p_pair**2*qp - 2*p_pair**2*r + p_pair - 2*qp + 2)/8,
   (pmp)*qp**2*t*(-4*p_pair**4*qp*r**2 + 4*p_pair**3*qp*r + 4*p_pair**3*r**2 - 2*p_pair**2*qp - 4*p_pair**2*r + 2*p_pair - qp + 1)/12,
   (pmp)*qp**2*t*(-p_pair**2*qp + p_pair - 2*qp + 2)/8]],
 [6,
  [(pdm)*t*(q - 1)**2 + (pmp)*t*(qp - 1)**3*(3*qp - 4)/4 + (ppc)*t*(q - 1)*(2*q*vp**2 - 2*q*vp + 3*q - 2*vp**2 + 2*vp - 4)/4 + (q - 1)**2*(-(pdm)*t - (pmp)*t - (ppc)*t + 1),
   (pdm)*t*(q - 1)**2*(-p_pair**2*rp**2 + p_pair*rp + 4)/8 + (pmp)*t*(qp - 1)**2*(-p_pair**4*qp**2*r**2 + p_pair**3*qp**2*r + 2*p_pair**3*qp*r**2 - 2*p_pair**2*qp*r - p_pair**2*r**2 + 2*p_pair*qp**2 - 2*p_pair*qp + p_pair*r + qp**2 - 5*qp + 4)/8 + (ppc)*t*(-2*p_pair**6*q**2*r**2*vp**2 + 2*p_pair**5*q**2*r**2*vp + 2*p_pair**5*q**2*r*vp**2 + 4*p_pair**5*q*r**2*vp**2 - p_pair**4*q**2*r**2 - 2*p_pair**4*q**2*r*vp - 4*p_pair**4*q*r**2*vp - 4*p_pair**4*q*r*vp**2 - 2*p_pair**4*r**2*vp**2 + p_pair**3*q**2*r + 2*p_pair**3*q*r**2 + 4*p_pair**3*q*r*vp + 2*p_pair**3*r**2*vp + 2*p_pair**3*r*vp**2 - 2*p_pair**2*q*r - p_pair**2*r**2 - 2*p_pair**2*r*vp + 2*p_pair*q**2 - 2*p_pair*q + p_pair*r + 2*q**2*vp**2 - 2*q**2*vp + q**2 - 4*q*vp**2 + 4*q*vp - 5*q + 2*vp**2 - 2*vp + 4)/8 - (q - 1)*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/2,
   (pdm)*t*(q - 1)**2*(-p_pair**2*rp**2 + p_pair*rp + 1)/6 + (pmp)*t*(qp - 1)**2*(-4*p_pair**4*qp**2*r**2 + 4*p_pair**3*qp**2*r + 8*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 - 8*p_pair**2*qp*r - 4*p_pair**2*r**2 - 4*p_pair*qp + 4*p_pair*r + qp**2 - 3*qp + 4)/24 + (ppc)*t*(-8*p_pair**6*q**2*r**2*vp**2 + 8*p_pair**5*q**2*r**2*vp + 8*p_pair**5*q**2*r*vp**2 + 16*p_pair**5*q*r**2*vp**2 - 4*p_pair**4*q**2*r**2 - 8*p_pair**4*q**2*r*vp - 16*p_pair**4*q*r**2*vp - 16*p_pair**4*q*r*vp**2 - 8*p_pair**4*r**2*vp**2 + 4*p_pair**3*q**2*r + 8*p_pair**3*q*r**2 + 16*p_pair**3*q*r*vp + 8*p_pair**3*r**2*vp + 8*p_pair**3*r*vp**2 + 2*p_pair**2*q**2 - 8*p_pair**2*q*r - 4*p_pair**2*r**2 - 8*p_pair**2*r*vp - 4*p_pair*q + 4*p_pair*r + 2*q**2*vp**2 - 2*q**2*vp + q**2 - 4*q*vp**2 + 4*q*vp - 3*q + 2*vp**2 - 2*vp + 4)/24 + (p_pair*q - 1)**2*(-(pdm)*t - (pmp)*t - (ppc)*t + 1)/6,
   3*(pdm)*t*(q - 1)**2/4 + (pmp)*t*(qp - 1)**2*(p_pair**2*qp**2 + 6*p_pair*qp**2 - 9*p_pair*qp + 2*qp**2 - 12*qp + 12)/16 + (ppc)*t*(2*p_pair**4*q**2*vp**2 - 2*p_pair**3*q**2*vp - 4*p_pair**3*q*vp**2 + p_pair**2*q**2 + 4*p_pair**2*q*vp + 2*p_pair**2*vp**2 + 6*p_pair*q**2 - 9*p_pair*q - 2*p_pair*vp + 4*q**2*vp**2 - 4*q**2*vp + 2*q**2 - 8*q*vp**2 + 8*q*vp - 12*q + 4*vp**2 - 4*vp + 12)/16 - 3*(q - 1)*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/4]],
 [7,
  [-2*(pdm)*q*t*(q - 1) - 3*(pmp)*qp*t*(qp - 2)*(qp - 1)**2/2 - (ppc)*q*t*(q - 1) + 2*q*(q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1),
   -(pdm)*q*t*(q - 1)*(-p_pair**2*rp**2 + p_pair*rp + 4)/4 - (pmp)*qp*t*(qp - 1)*(-p_pair**4*qp**2*r**2 + p_pair**3*qp**2*r + 2*p_pair**3*qp*r**2 - 2*p_pair**2*qp*r - p_pair**2*r**2 + 2*p_pair*qp**2 - 4*p_pair*qp + p_pair*r + p_pair + qp**2 - 5*qp + 5)/4 + (ppc)*q*t*(-2*p_pair*q + p_pair + 1)/4 - q*(-2*p_pair*q + p_pair + 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/2,
   -(pdm)*q*t*(q - 1)*(-p_pair**2*rp**2 + p_pair*rp + 1)/3 - (pmp)*qp*t*(qp - 1)*(-4*p_pair**4*qp**2*r**2 + 4*p_pair**3*qp**2*r + 8*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 - 8*p_pair**2*qp*r - 2*p_pair**2*qp - 4*p_pair**2*r**2 - 4*p_pair*qp + 4*p_pair*r + 2*p_pair + qp**2 - 3*qp + 4)/12 - (ppc)*p_pair*q*t*(p_pair*q - 1)/6 + p_pair*q*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/3,
   -3*(pdm)*q*t*(q - 1)/2 - (pmp)*qp*t*(qp - 1)*(p_pair**2*qp**2 + 6*p_pair*qp**2 - 15*p_pair*qp + 3*p_pair + 2*qp**2 - 12*qp + 15)/8 + 3*(ppc)*q*t*(-2*p_pair*q + p_pair + 1)/8 - 3*q*(-2*p_pair*q + p_pair + 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/4]],
 [8,
  [t*((pmp)*qp*(3*qp**3 - qp**2 - qp + 1) + (ppc)*q*(2*q*vp**2 - 2*q*vp + q + 1))/4,
   t*((pmp)*qp*(-p_pair**4*qp**3*r**2 + 2*p_pair**4*qp**2*r**2 - p_pair**4*qp*r**2 + p_pair**3*qp**3*r - 2*p_pair**3*qp**2*r + p_pair**3*qp*r + 2*p_pair*qp**3 + qp**3 - qp**2 - qp + 1) + (ppc)*q*(-2*p_pair**6*q*r**2*vp**2 + 2*p_pair**5*q*r**2*vp + 2*p_pair**5*q*r*vp**2 - p_pair**4*q*r**2 - 2*p_pair**4*q*r*vp + p_pair**3*q*r + 2*q*vp**2 - 2*q*vp + q + 1))/8,
   t*((pmp)*qp*(-4*p_pair**4*qp**3*r**2 + 8*p_pair**4*qp**2*r**2 - 4*p_pair**4*qp*r**2 + 4*p_pair**3*qp**3*r - 8*p_pair**3*qp**2*r + 4*p_pair**3*qp*r + 2*p_pair**2*qp**3 + qp**3 - qp**2 - qp + 1) + (ppc)*q*(-8*p_pair**6*q*r**2*vp**2 + 8*p_pair**5*q*r**2*vp + 8*p_pair**5*q*r*vp**2 - 4*p_pair**4*q*r**2 - 8*p_pair**4*q*r*vp + 4*p_pair**3*q*r + 2*q*vp**2 - 2*q*vp + q + 1))/24,
   t*((pmp)*qp*(p_pair**2*qp**3 - 2*p_pair**2*qp**2 + p_pair**2*qp + 6*p_pair*qp**3 + p_pair*qp**2 - 2*p_pair*qp + p_pair + 2*qp**3 - 2*qp**2 - 2*qp + 2) + (ppc)*q*(2*p_pair**4*q*vp**2 - 2*p_pair**3*q*vp + p_pair**2*q + p_pair + 4*q*vp**2 - 4*q*vp + 2*q + 2))/16]],
 [9,
  [3*(pmp)*qp**2*t*(-qp**2 + 1)/2,
   (pmp)*qp**2*t*(p_pair**4*qp**2*r**2 - p_pair**4*qp*r**2 - p_pair**3*qp**2*r + p_pair**3*qp*r - 2*p_pair*qp**2 + p_pair - qp**2 + 2)/4,
   (pmp)*qp**2*t*(4*p_pair**4*qp**2*r**2 - 4*p_pair**4*qp*r**2 - 4*p_pair**3*qp**2*r + 4*p_pair**3*qp*r - 2*p_pair**2*qp**2 + 2*p_pair - qp**2 + 1)/12,
   (pmp)*qp**2*t*(-p_pair**2*qp**2 + p_pair**2*qp - 6*p_pair*qp**2 - p_pair*qp + 4*p_pair - 2*qp**2 + 5)/8]],
 [10,
  [-(pmp)*q**2*t + 3*(pmp)*qp**4*t/2 - 5*(pmp)*qp**3*t/2 + 3*(pmp)*qp**2*t/2 - (ppc)*q**2*t/2 + q**2,
   (pdm)*q**2*t*(-p_pair**2*rp**2 + p_pair*rp + 4)/8 + (pmp)*qp**2*t*(-2*p_pair**4*qp**2*r**2 + 2*p_pair**3*qp**2*r + 2*p_pair**3*qp*r**2 - 2*p_pair**2*qp*r - p_pair**2*r**2 + 4*p_pair*qp**2 - 6*p_pair*qp + p_pair*r + 2*p_pair + 2*qp**2 - 4*qp + 4)/8 + (ppc)*p_pair*q**2*t/4 - p_pair*q**2*((pdm)*t + (pmp)*t + (ppc)*t - 1)/2,
   (pdm)*q**2*t*(-p_pair**2*rp**2 + p_pair*rp + 1)/6 + (pmp)*qp**2*t*(-4*p_pair**4*qp**2*r**2 + 4*p_pair**3*qp**2*r + 4*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 - 4*p_pair**2*qp*r - 2*p_pair**2*qp - 2*p_pair**2*r**2 + p_pair**2 - 2*p_pair*qp + 2*p_pair*r + qp**2 - qp + 2)/12 + (ppc)*p_pair**2*q**2*t/12 + p_pair**2*q**2*(-(pdm)*t - (pmp)*t - (ppc)*t + 1)/6,
   3*(pdm)*q**2*t/4 + (pmp)*qp**2*t*(p_pair**2*qp**2 + 6*p_pair*qp**2 - 10*p_pair*qp + 3*p_pair + 2*qp**2 - 5*qp + 6)/8 + 3*(ppc)*p_pair*q**2*t/8 - 3*p_pair*q**2*((pdm)*t + (pmp)*t + (ppc)*t - 1)/4]],
 [11,
  [0,
   (ppc)*p_pair**2*q*t*vp*(p_pair*q - 1)*(p_pair*vp - 1)*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/4,
   (ppc)*p_pair**2*q*t*vp*(p_pair*q - 1)*(p_pair*vp - 1)*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/3,
   (ppc)*p_pair**2*q*t*vp*(p_pair*q - 1)*(p_pair*vp - 1)/8]],
 [12,
  [0,
   p_pair**2*r*t*(p_pair*r - 1)*(-(pmp)*qp*(qp - 1)*(p_pair*qp - 1) + (ppc)*q*(p_pair*q - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/4,
   p_pair**2*r*t*(p_pair*r - 1)*(-(pmp)*qp*(qp - 1)*(p_pair*qp - 1) + (ppc)*q*(p_pair*q - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/3,
   p_pair*t*((pmp)*qp*(qp - 1)*(p_pair*qp - 1) - (ppc)*q*(p_pair*q - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/16]],
 [13,
  [0,
   -(pdm)*q*t*(q - 1)*(p_pair**2*rp**2 - p_pair*rp + 2)/4 - (pmp)*qp*t*(qp - 1)*(2*p_pair**4*qp**2*r**2 - 2*p_pair**3*qp**2*r - 4*p_pair**3*qp*r**2 + p_pair**2*qp**2 + 4*p_pair**2*qp*r + 2*p_pair**2*r**2 + 2*p_pair*qp**2 - 7*p_pair*qp - 2*p_pair*r + 2*p_pair - 2*qp + 4)/8 - (ppc)*p_pair*q*t*(q - 1)/4 + p_pair*q*(q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/2,
   -(pdm)*q*t*(q - 1)*(p_pair**2*rp**2 - p_pair*rp + 2)/3 - (pmp)*qp*t*(qp - 1)*(2*p_pair**4*qp**2*r**2 - 2*p_pair**3*qp**2*r - 4*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 + 4*p_pair**2*qp*r - p_pair**2*qp + 2*p_pair**2*r**2 - 5*p_pair*qp - 2*p_pair*r + p_pair + qp**2 - 3*qp + 5)/6 - (ppc)*q*t*(p_pair**2*q - p_pair + q - 1)/6 + q*((pdm)*t + (pmp)*t + (ppc)*t - 1)*(p_pair**2*q - p_pair + q - 1)/3,
   -(pdm)*q*t*(q - 1)/4 - (pmp)*qp*t*(qp - 1)*(p_pair**2*qp**2 + 2*p_pair*qp**2 - 7*p_pair*qp + 2*p_pair - 2*qp + 4)/16 - (ppc)*p_pair*q*t*(q - 1)/8 + p_pair*q*(q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/4]],
 [14,
  [0,
   (pmp)*qp**2*t*(-2*p_pair**4*qp**2*r**2 + 2*p_pair**4*qp*r**2 + 2*p_pair**3*qp**2*r - 2*p_pair**3*qp*r - p_pair**2*qp**2 + p_pair**2*qp - 2*p_pair*qp**2 - p_pair*qp + p_pair + 2)/8,
   (pmp)*qp**2*t*(-2*p_pair**4*qp**2*r**2 + 2*p_pair**4*qp*r**2 + 2*p_pair**3*qp**2*r - 2*p_pair**3*qp*r - 2*p_pair**2*qp**2 + p_pair**2*qp - p_pair*qp + 2*p_pair - qp**2 + 1)/6,
   (pmp)*qp**2*t*(-p_pair**2*qp**2 + p_pair**2*qp - 2*p_pair*qp**2 - p_pair*qp + p_pair + 2)/16]],
 [15,
  [0,
   (ppc)*p_pair**2*r*t*vp*(p_pair*q - 1)**2*(p_pair*r - 1)*(p_pair*vp - 1)/2,
   2*(ppc)*p_pair**2*r*t*vp*(p_pair*q - 1)**2*(p_pair*r - 1)*(p_pair*vp - 1)/3,
   -(ppc)*p_pair*t*vp*(p_pair*q - 1)**2*(p_pair*vp - 1)/8]],
 [16,
  [0,
   (ppc)*p_pair**4*q**2*r*t*vp*(p_pair*r - 1)*(p_pair*vp - 1)/2,
   2*(ppc)*p_pair**4*q**2*r*t*vp*(p_pair*r - 1)*(p_pair*vp - 1)/3,
   (ppc)*p_pair**3*q**2*t*vp*(-p_pair*vp + 1)/8]],
 [17,
  [0,
   (ppc)*p_pair**2*q*t*vp*(p_pair*q - 1)*(p_pair*vp - 1)*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/4,
   (ppc)*p_pair**2*q*t*vp*(p_pair*q - 1)*(p_pair*vp - 1)*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/3,
   (ppc)*p_pair**2*q*t*vp*(p_pair*q - 1)*(p_pair*vp - 1)/8]],
 [18, [0, 0, 0, 0]],
 [19,
  [0,
   p_pair**2*r*t*(p_pair*r - 1)*(-(pmp)*qp*(qp - 1)*(p_pair*qp - 1) + (ppc)*q*(p_pair*q - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/4,
   p_pair**2*r*t*(p_pair*r - 1)*(-(pmp)*qp*(qp - 1)*(p_pair*qp - 1) + (ppc)*q*(p_pair*q - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/3,
   p_pair*t*((pmp)*qp*(qp - 1)*(p_pair*qp - 1) - (ppc)*q*(p_pair*q - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/16]],
 [20,
  [0,
   (pmp)*p_pair**2*qp**2*r*t*(p_pair*qp - 1)*(p_pair*r - 1)/2,
   2*(pmp)*p_pair**2*qp**2*r*t*(p_pair*qp - 1)*(p_pair*r - 1)/3,
   (pmp)*p_pair*qp**2*t*(-p_pair*qp + 1)/8]],
 [21,
  [0,
   (pdm)*t*(q - 1)**2*(p_pair**2*rp**2 - p_pair*rp + 2)/4 + (pmp)*t*(qp - 1)**2*(p_pair*qp - 1)*(2*p_pair**3*qp*r**2 - 2*p_pair**2*qp*r - 2*p_pair**2*r**2 + p_pair*qp + 2*p_pair*r + 2*qp - 4)/8 + (ppc)*t*(p_pair*q - 1)*(4*p_pair**5*q*r**2*vp**2 - 4*p_pair**4*q*r**2*vp - 4*p_pair**4*q*r*vp**2 - 4*p_pair**4*r**2*vp**2 + 2*p_pair**3*q*r**2 + 4*p_pair**3*q*r*vp + 2*p_pair**3*q*vp**2 + 4*p_pair**3*r**2*vp + 4*p_pair**3*r*vp**2 - 2*p_pair**2*q*r - 2*p_pair**2*q*vp - 2*p_pair**2*r**2 - 4*p_pair**2*r*vp - 2*p_pair**2*vp**2 + p_pair*q + 2*p_pair*r + 2*p_pair*vp + 2*q - 4)/8 - (q - 1)*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/2,
   (pdm)*t*(q - 1)**2*(p_pair**2*rp**2 - p_pair*rp + 2)/3 + (pmp)*t*(qp - 1)**2*(2*p_pair**4*qp**2*r**2 - 2*p_pair**3*qp**2*r - 4*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 + 4*p_pair**2*qp*r + 2*p_pair**2*r**2 - 5*p_pair*qp - 2*p_pair*r + qp**2 - 2*qp + 4)/6 + (ppc)*t*(4*p_pair**6*q**2*r**2*vp**2 - 4*p_pair**5*q**2*r**2*vp - 4*p_pair**5*q**2*r*vp**2 - 8*p_pair**5*q*r**2*vp**2 + 2*p_pair**4*q**2*r**2 + 4*p_pair**4*q**2*r*vp + 2*p_pair**4*q**2*vp**2 + 8*p_pair**4*q*r**2*vp + 8*p_pair**4*q*r*vp**2 + 4*p_pair**4*r**2*vp**2 - 2*p_pair**3*q**2*r - 2*p_pair**3*q**2*vp - 4*p_pair**3*q*r**2 - 8*p_pair**3*q*r*vp - 4*p_pair**3*q*vp**2 - 4*p_pair**3*r**2*vp - 4*p_pair**3*r*vp**2 + 2*p_pair**2*q**2 + 4*p_pair**2*q*r + 4*p_pair**2*q*vp + 2*p_pair**2*r**2 + 4*p_pair**2*r*vp + 2*p_pair**2*vp**2 - 5*p_pair*q - 2*p_pair*r - 2*p_pair*vp + q**2 - 2*q + 4)/6 - ((pdm)*t + (pmp)*t + (ppc)*t - 1)*(p_pair**2*q**2 - 2*p_pair*q + q**2 - 2*q + 2)/3,
   (pdm)*t*(q - 1)**2/4 + (pmp)*t*(qp - 1)**2*(p_pair*qp - 1)*(p_pair*qp + 2*qp - 4)/16 + (ppc)*t*(p_pair*q - 1)*(2*p_pair**3*q*vp**2 - 2*p_pair**2*q*vp - 2*p_pair**2*vp**2 + p_pair*q + 2*p_pair*vp + 2*q - 4)/16 - (q - 1)*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/4]],
 [22,
  [0,
   -(pdm)*q*t*(q - 1)*(p_pair**2*rp**2 - p_pair*rp + 2)/4 - (pmp)*qp*t*(qp - 1)*(p_pair*qp - 1)*(2*p_pair**3*qp*r**2 - 2*p_pair**2*qp*r - 2*p_pair**2*r**2 + p_pair*qp + 2*p_pair*r + 2*qp - 6)/8 - (ppc)*q*t*(p_pair*q - 1)/4 + q*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/2,
   -(pdm)*q*t*(q - 1)*(p_pair**2*rp**2 - p_pair*rp + 2)/3 - (pmp)*qp*t*(qp - 1)*(2*p_pair**4*qp**2*r**2 - 2*p_pair**3*qp**2*r - 4*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 + 4*p_pair**2*qp*r - p_pair**2*qp + 2*p_pair**2*r**2 - 5*p_pair*qp - 2*p_pair*r + p_pair + qp**2 - 3*qp + 5)/6 - (ppc)*q*t*(p_pair**2*q - p_pair + q - 1)/6 + q*((pdm)*t + (pmp)*t + (ppc)*t - 1)*(p_pair**2*q - p_pair + q - 1)/3,
   -(pdm)*q*t*(q - 1)/4 - (pmp)*qp*t*(qp - 1)*(p_pair*qp - 1)*(p_pair*qp + 2*qp - 6)/16 - (ppc)*q*t*(p_pair*q - 1)/8 + q*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/4]],
 [23,
  [0,
   p_pair*t*((pmp)*qp*(2*p_pair**3*qp**3*r**2 - 4*p_pair**3*qp**2*r**2 + 2*p_pair**3*qp*r**2 - 2*p_pair**2*qp**3*r + 4*p_pair**2*qp**2*r - 2*p_pair**2*qp*r + p_pair*qp**3 - 2*p_pair*qp**2 + p_pair*qp + 2*qp**3 + qp**2 - 2*qp + 1) + (ppc)*q*(4*p_pair**5*q*r**2*vp**2 - 4*p_pair**4*q*r**2*vp - 4*p_pair**4*q*r*vp**2 + 2*p_pair**3*q*r**2 + 4*p_pair**3*q*r*vp + 2*p_pair**3*q*vp**2 - 2*p_pair**2*q*r - 2*p_pair**2*q*vp + p_pair*q + 1))/8,
   t*((pmp)*qp*(2*p_pair**4*qp**3*r**2 - 4*p_pair**4*qp**2*r**2 + 2*p_pair**4*qp*r**2 - 2*p_pair**3*qp**3*r + 4*p_pair**3*qp**2*r - 2*p_pair**3*qp*r + 2*p_pair**2*qp**3 - 2*p_pair**2*qp**2 + p_pair**2*qp + p_pair*qp**2 - 2*p_pair*qp + p_pair + qp**3) + (ppc)*p_pair*q*(4*p_pair**5*q*r**2*vp**2 - 4*p_pair**4*q*r**2*vp - 4*p_pair**4*q*r*vp**2 + 2*p_pair**3*q*r**2 + 4*p_pair**3*q*r*vp + 2*p_pair**3*q*vp**2 - 2*p_pair**2*q*r - 2*p_pair**2*q*vp + p_pair*q + 1))/6,
   p_pair*t*((pmp)*qp*(p_pair*qp**3 - 2*p_pair*qp**2 + p_pair*qp + 2*qp**3 + qp**2 - 2*qp + 1) + (ppc)*q*(2*p_pair**3*q*vp**2 - 2*p_pair**2*q*vp + p_pair*q + 1))/16]],
 [24,
  [0,
   -(pmp)*p_pair*qp**2*t*(qp - 1)*(2*p_pair**3*qp*r**2 - 2*p_pair**2*qp*r + p_pair*qp + 2*qp + 3)/8,
   (pmp)*qp**2*t*(-2*p_pair**4*qp**2*r**2 + 2*p_pair**4*qp*r**2 + 2*p_pair**3*qp**2*r - 2*p_pair**3*qp*r - 2*p_pair**2*qp**2 + p_pair**2*qp - p_pair*qp + 2*p_pair - qp**2 + 1)/6,
   -(pmp)*p_pair*qp**2*t*(qp - 1)*(p_pair*qp + 2*qp + 3)/16]],
 [25,
  [0,
   (pdm)*q**2*t*(p_pair**2*rp**2 - p_pair*rp + 2)/4 + (pmp)*qp**2*t*(2*p_pair**4*qp**2*r**2 - 2*p_pair**3*qp**2*r - 2*p_pair**3*qp*r**2 + p_pair**2*qp**2 + 2*p_pair**2*qp*r + p_pair**2*r**2 + 2*p_pair*qp**2 - 4*p_pair*qp - p_pair*r + p_pair - qp + 2)/4 + (ppc)*p_pair*q**2*t/4 - p_pair*q**2*((pdm)*t + (pmp)*t + (ppc)*t - 1)/2,
   (pdm)*q**2*t*(p_pair**2*rp**2 - p_pair*rp + 2)/3 + (pmp)*qp**2*t*(4*p_pair**4*qp**2*r**2 - 4*p_pair**3*qp**2*r - 4*p_pair**3*qp*r**2 + 4*p_pair**2*qp**2 + 4*p_pair**2*qp*r - 2*p_pair**2*qp + 2*p_pair**2*r**2 + p_pair**2 - 4*p_pair*qp - 2*p_pair*r + 2*qp**2 - 4*qp + 5)/6 + (ppc)*q**2*t*(p_pair**2 + 1)/6 - q**2*(p_pair**2 + 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/3,
   (pdm)*q**2*t/4 + (pmp)*qp**2*t*(p_pair**2*qp**2 + 2*p_pair*qp**2 - 4*p_pair*qp + p_pair - qp + 2)/8 + (ppc)*p_pair*q**2*t/8 - p_pair*q**2*((pdm)*t + (pmp)*t + (ppc)*t - 1)/4]],
 [26,
  [0,
   -(ppc)*p_pair*t*vp*(p_pair*q - 1)**2*(p_pair*vp - 1)*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/8,
   (ppc)*t*vp*(-4*p_pair**6*q**2*r**2*vp + 4*p_pair**5*q**2*r**2 + 4*p_pair**5*q**2*r*vp + 8*p_pair**5*q*r**2*vp - 4*p_pair**4*q**2*r - 2*p_pair**4*q**2*vp - 8*p_pair**4*q*r**2 - 8*p_pair**4*q*r*vp - 4*p_pair**4*r**2*vp + 2*p_pair**3*q**2 + 8*p_pair**3*q*r + 4*p_pair**3*q*vp + 4*p_pair**3*r**2 + 4*p_pair**3*r*vp - 4*p_pair**2*q - 4*p_pair**2*r - 2*p_pair**2*vp + 2*p_pair - q**2*vp + q**2 + 2*q*vp - 2*q - vp + 1)/12,
   0]],
 [27,
  [0,
   -(ppc)*p_pair**3*q**2*t*vp*(p_pair*vp - 1)*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/8,
   (ppc)*q**2*t*vp*(-4*p_pair**6*r**2*vp + 4*p_pair**5*r**2 + 4*p_pair**5*r*vp - 4*p_pair**4*r - 2*p_pair**4*vp + 2*p_pair**3 - vp + 1)/12,
   0]],
 [28,
  [0,
   -(ppc)*p_pair**3*q*r*t*vp*(p_pair*q - 1)*(p_pair*r - 1)*(p_pair*vp - 1)/2,
   (ppc)*q*t*vp*(-4*p_pair**6*q*r**2*vp + 4*p_pair**5*q*r**2 + 4*p_pair**5*q*r*vp + 4*p_pair**5*r**2*vp - 4*p_pair**4*q*r - 4*p_pair**4*r**2 - 4*p_pair**4*r*vp + 4*p_pair**3*r + q*vp - q - vp + 1)/6,
   0]],
 [29, [0, 0, 0, 0]],
 [30,
  [0,
   p_pair*t*((pmp)*qp*(qp - 1)*(p_pair*qp - 1) - (ppc)*q*(p_pair*q - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/8,
   t*((pmp)*qp*(qp - 1)*(4*p_pair**4*qp*r**2 - 4*p_pair**3*qp*r - 4*p_pair**3*r**2 + 2*p_pair**2*qp + 4*p_pair**2*r - 2*p_pair + qp - 1) - (ppc)*q*(8*p_pair**6*q*r**2*vp**2 - 8*p_pair**5*q*r**2*vp - 8*p_pair**5*q*r*vp**2 - 8*p_pair**5*r**2*vp**2 + 4*p_pair**4*q*r**2 + 8*p_pair**4*q*r*vp + 4*p_pair**4*q*vp**2 + 8*p_pair**4*r**2*vp + 8*p_pair**4*r*vp**2 - 4*p_pair**3*q*r - 4*p_pair**3*q*vp - 4*p_pair**3*r**2 - 8*p_pair**3*r*vp - 4*p_pair**3*vp**2 + 2*p_pair**2*q + 4*p_pair**2*r + 4*p_pair**2*vp - 2*p_pair + 2*q*vp**2 - 2*q*vp + q - 2*vp**2 + 2*vp - 1))/12,
   0]],
 [31,
  [0,
   -(pmp)*p_pair*qp**2*t*(p_pair*qp - 1)*(2*p_pair**2*r**2 - 2*p_pair*r + 1)/8,
   (pmp)*qp**2*t*(-4*p_pair**4*qp*r**2 + 4*p_pair**3*qp*r + 4*p_pair**3*r**2 - 2*p_pair**2*qp - 4*p_pair**2*r + 2*p_pair - qp + 1)/12,
   0]],
 [32,
  [0,
   -p_pair*t*((pdm)*rp*(q - 1)**2*(p_pair*rp - 1) + (pmp)*r*(qp - 1)**2*(p_pair*qp - 1)**2*(p_pair*r - 1) + (ppc)*r*(p_pair*q - 1)**2*(p_pair*r - 1)*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/8,
   (pdm)*t*(q - 1)**2*(-p_pair**2*rp**2 + p_pair*rp + 1)/6 + (pmp)*t*(qp - 1)**2*(-4*p_pair**4*qp**2*r**2 + 4*p_pair**3*qp**2*r + 8*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 - 8*p_pair**2*qp*r - 4*p_pair**2*r**2 - 4*p_pair*qp + 4*p_pair*r + qp**2 - 3*qp + 4)/24 + (ppc)*t*(-8*p_pair**6*q**2*r**2*vp**2 + 8*p_pair**5*q**2*r**2*vp + 8*p_pair**5*q**2*r*vp**2 + 16*p_pair**5*q*r**2*vp**2 - 4*p_pair**4*q**2*r**2 - 8*p_pair**4*q**2*r*vp - 16*p_pair**4*q*r**2*vp - 16*p_pair**4*q*r*vp**2 - 8*p_pair**4*r**2*vp**2 + 4*p_pair**3*q**2*r + 8*p_pair**3*q*r**2 + 16*p_pair**3*q*r*vp + 8*p_pair**3*r**2*vp + 8*p_pair**3*r*vp**2 + 2*p_pair**2*q**2 - 8*p_pair**2*q*r - 4*p_pair**2*r**2 - 8*p_pair**2*r*vp - 4*p_pair*q + 4*p_pair*r + 2*q**2*vp**2 - 2*q**2*vp + q**2 - 4*q*vp**2 + 4*q*vp - 3*q + 2*vp**2 - 2*vp + 4)/24 + (p_pair*q - 1)**2*(-(pdm)*t - (pmp)*t - (ppc)*t + 1)/6,
   0]],
 [33,
  [0,
   p_pair*t*((pdm)*q*rp*(q - 1)*(p_pair*rp - 1) + (pmp)*qp*r*(qp - 1)*(p_pair*qp - 1)**2*(p_pair*r - 1))/4,
   -(pdm)*q*t*(q - 1)*(-p_pair**2*rp**2 + p_pair*rp + 1)/3 - (pmp)*qp*t*(qp - 1)*(-4*p_pair**4*qp**2*r**2 + 4*p_pair**3*qp**2*r + 8*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 - 8*p_pair**2*qp*r - 2*p_pair**2*qp - 4*p_pair**2*r**2 - 4*p_pair*qp + 4*p_pair*r + 2*p_pair + qp**2 - 3*qp + 4)/12 - (ppc)*p_pair*q*t*(p_pair*q - 1)/6 + p_pair*q*(p_pair*q - 1)*((pdm)*t + (pmp)*t + (ppc)*t - 1)/3,
   0]],
 [34,
  [0,
   -p_pair**3*r*t*(p_pair*r - 1)*((pmp)*qp**2*(qp - 1)**2 + (ppc)*q**2*(2*p_pair**2*vp**2 - 2*p_pair*vp + 1))/8,
   t*((pmp)*qp*(-4*p_pair**4*qp**3*r**2 + 8*p_pair**4*qp**2*r**2 - 4*p_pair**4*qp*r**2 + 4*p_pair**3*qp**3*r - 8*p_pair**3*qp**2*r + 4*p_pair**3*qp*r + 2*p_pair**2*qp**3 + qp**3 - qp**2 - qp + 1) + (ppc)*q*(-8*p_pair**6*q*r**2*vp**2 + 8*p_pair**5*q*r**2*vp + 8*p_pair**5*q*r*vp**2 - 4*p_pair**4*q*r**2 - 8*p_pair**4*q*r*vp + 4*p_pair**3*q*r + 2*q*vp**2 - 2*q*vp + q + 1))/24,
   0]],
 [35,
  [0,
   (pmp)*p_pair**3*qp**3*r*t*(qp - 1)*(p_pair*r - 1)/4,
   (pmp)*qp**2*t*(4*p_pair**4*qp**2*r**2 - 4*p_pair**4*qp*r**2 - 4*p_pair**3*qp**2*r + 4*p_pair**3*qp*r - 2*p_pair**2*qp**2 + 2*p_pair - qp**2 + 1)/12,
   0]],
 [36,
  [0,
   -p_pair*t*((pdm)*q**2*rp*(p_pair*rp - 1) + (pmp)*qp**2*r*(p_pair*r - 1)*(2*p_pair**2*qp**2 - 2*p_pair*qp + 1))/8,
   (pdm)*q**2*t*(-p_pair**2*rp**2 + p_pair*rp + 1)/6 + (pmp)*qp**2*t*(-4*p_pair**4*qp**2*r**2 + 4*p_pair**3*qp**2*r + 4*p_pair**3*qp*r**2 + 2*p_pair**2*qp**2 - 4*p_pair**2*qp*r - 2*p_pair**2*qp - 2*p_pair**2*r**2 + p_pair**2 - 2*p_pair*qp + 2*p_pair*r + qp**2 - qp + 2)/12 + (ppc)*p_pair**2*q**2*t/12 + p_pair**2*q**2*(-(pdm)*t - (pmp)*t - (ppc)*t + 1)/6,
   0]]]

In [11]:
u_geno.index([[A1, B1, G], [A1, B1, G], [A2, B2, g], [A2, B2, g]])

35

In [12]:
geno_num = 35
Gg_geno = 2

multinom_vec = []

for i in range(len(trans_matrix[geno_num])):
    tmp = trans_matrix[geno_num][i]
    for j in range(37):
        if tmp != 0:
            tmp = tmp.subs(p(j),mode_prob[j][1][Gg_geno]).subs(rp,(1 - (1 - 2 * r)**(1/2))/2)\
                                                           .subs(qp,(1 - (1 - 2 * q)**(1/2))/2)\
                                                           .subs(vp,(1 - (1 - 2 * v)**(1/2))/2)
    multinom_vec.append(tmp)

In [13]:
def MLFunction_switch_p_pair(params,n):
            
    tmp = []
    for i in range(len(multinom_vec)):
        loop_tmp = multinom_vec[i].subs(t,2/3).subs(r,1/2).subs(q,1/2).subs(v,1/2) \
                    .subs(ppc,params[0]).subs(pdm,params[0]).subs(pmp,params[0]).subs(p_pair,params[1])
        if loop_tmp > 0: 
            tmp.append(n[i] * m.log(loop_tmp))
        else:
            tmp.append(0)
    
    likelih = -sum([tmp[i] for i in range(len(multinom_vec))])
    
    return likelih

In [14]:
multinom_vec_num = []

for i in range(len(multinom_vec)):
    if multinom_vec[i] == 0:
        multinom_vec_num.append(0)
    else:
        multinom_vec_num.append(multinom_vec[i].subs(t,2/3).subs(r,0.5).subs(q,0.5).subs(v,0.5).subs(ppc,1/4) \
                       .subs(pdm,1/4).subs(pmp,1/4).subs(p_pair,0.5))

In [15]:
def ML_rep():
    n = np.random.multinomial(50000, multinom_vec_num, size=1)[0]
    results = minimize(MLFunction_switch_p_pair, np.array([0.125,0.25]),method = 'L-BFGS-B',\
                      bounds = [(0 + 1e-6,0.5),(0 +1e-6,1 - 1e-6)], args = (n))
    return results

In [16]:
num_cores = 24
ml_results = Parallel(n_jobs=num_cores)(delayed(ML_rep)() for i in range(100))

In [17]:
np.average([list(ml_results[i].x)[0] for i in range(len(ml_results))])

0.24968756879798948

In [18]:
np.std([list(ml_results[i].x)[0] for i in range(len(ml_results))])

0.009563178067028017

In [19]:
np.average([list(ml_results[i].x)[1] for i in range(len(ml_results))])

0.500033952271906

In [20]:
np.std([list(ml_results[i].x)[1] for i in range(len(ml_results))])

0.013715526267896749

In [21]:
[list(ml_results[i].x) for i in range(len(ml_results))]

[[0.2418834495445998, 0.5049945970330083],
 [0.25936691923315275, 0.4948305742858277],
 [0.25415660063205986, 0.5074256908008028],
 [0.24537957861439758, 0.5050131879162684],
 [0.2491455713332143, 0.5022686564705142],
 [0.27514484432213654, 0.4606472820997973],
 [0.23006370658594763, 0.522571846749514],
 [0.24934092550997816, 0.5039519329343032],
 [0.23051256453599517, 0.5189077847307415],
 [0.2549616511414644, 0.5062883632697556],
 [0.25678044676823697, 0.49563853243009076],
 [0.24462843206418639, 0.5028651349196552],
 [0.2641590938112347, 0.48769936286189003],
 [0.24572309791100952, 0.5135249264451286],
 [0.24960087657643035, 0.49207158344440616],
 [0.23592697789758982, 0.5043173364762452],
 [0.2613835115960995, 0.48613114896896387],
 [0.24248103442767946, 0.4958285665193038],
 [0.2573810675407779, 0.4871946508026572],
 [0.2510988229007875, 0.5011219168996188],
 [0.24011407626436498, 0.500700758820806],
 [0.24678549195336327, 0.526304218496501],
 [0.23792026859547638, 0.5046898095488